# Self-driving JetBot Project Stub Code: Navigation Recording

In this notebook, we demonstrate how you can use a joypad/game controller to "teach" a jetbot how to drive on track.

## Take note

Your model training environment (the PC / the server with GPU) should use the same version of tensorflow as the jetbot.

## Library importing
First we initialize the environment.


In [1]:
from jetbot import Robot, Camera
import time
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets

import numpy as np

## Instantiting the robot and camera
We instantiate the robot object using the jetbot API.

Note that sometimes when we restart the kernels, the camera might not be re-initialized. We might need to run the following in the jetbot terminal.
```bash
$ sudo systemctl restart nvargus-daemon
```

In [7]:
robot = Robot()
camera = Camera.instance()

## Setting up the widgets for display and debugging

Next we setup the widgets for us to "drive" the jetbot and monitor its status.

In [2]:
controller = widgets.Controller(
    index=0,
)

image_widget = widgets.Image(format='jpeg', width=224, height=224)
buttonA_widget = widgets.FloatText(value=0, description='Button A')
buttonB_widget = widgets.FloatText(value=0, description='Button B')
axes0_widget = widgets.FloatText(value=0, description='Axis 0')
buttonR_widget = widgets.FloatText(value=0, description='Button R')
left_motor_widget = widgets.FloatText(value=0, description='Left motor')
right_motor_widget = widgets.FloatText(value=0, description='Right motor')
debug_widget = widgets.Textarea(value="",description='Debug mesg')

We then put the widgets in a container and display it. 

In [4]:
controller_container = widgets.HBox([image_widget,controller,widgets.VBox([axes0_widget,buttonA_widget, buttonB_widget, buttonR_widget, left_motor_widget, right_motor_widget, debug_widget])])

## Before we proceed

Please connect your controller to the PC/laptop where this notebook is currently open, and press a button (any button will do.)



## Stearing module

We use a python class to connect the joypad button press to the jetbot.

When the A button (accelerate) is pressed, the steering module will set the state of the jetbot motors to move forward.
When the B button (brake) and the A button (acceleration) are pressed at the same time, it will go backward.
When the left or right buttons from D-Pad (like a cross) is pressed, the jotbot goes left or right.

Feel free to adjust the steering to accomodate your need, e.g. you may implement a manual transition system.

In [5]:
# define a steearing traitlet to observe and combine the button events

SPEED = 0.3

class Steering(traitlets.HasTraits):
    left_motor = traitlets.Float()
    right_motor = traitlets.Float()
    def __init__(self, _acc, _brake, _left_right):
        
        self.acc = _acc
        self.brake = _brake
        self.left_right = _left_right
        super().__init__()
        self.set_notifiers()
        
    def func(self, change):
        speed = -(self.acc.value * SPEED) if self.brake.value > 0 else self.acc.value * SPEED
        self.right_motor = speed if self.left_right.value < (-0.5) else (speed * 0.5)
        self.left_motor = speed if self.left_right.value > (0.5) else (speed * 0.5)
        # print(change)
        
    def set_notifiers(self):
        traitlets.HasTraits.observe(self.acc, self.func, 'value')
        traitlets.HasTraits.observe(self.brake, self.func, 'value')
        traitlets.HasTraits.observe(self.left_right, self.func, 'value')

stearing = Steering(controller.buttons[1], controller.buttons[2], controller.axes[0])


## Linking up

We link up the camera, the joypad controller, the widget display, the steering module and the jetbot motors using the `traitlets.dlink()` method.

`traitlets.dlink(r1, r2)` create a one-direction observer on the value of `r1`, whenever it changes, it propogates the value to `r2` value asyncronously. This gives us a good abstraction for concurrency.

In [8]:
from jetbot import bgr8_to_jpeg
try: camera_link
except NameError: camera_link = None

if camera_link:
    camera_link.unlink()
    
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

try: button_a_label_link
except NameError: button_a_label_link = None

if button_a_label_link:
    button_a_label_link.unlink()

button_a_label_link = traitlets.dlink((controller.buttons[1], 'value'), (buttonA_widget, 'value'))

try: button_b_label_link
except NameError: button_b_label_link = None

if button_b_label_link:
    button_b_label_link.unlink()        

button_b_label_link = traitlets.dlink((controller.buttons[2], 'value'), (buttonB_widget, 'value'))
'''

try: button_r_label_link
except NameError: button_r_label_link = None

if button_r_label_link:
    button_r_label_link.unlink()        

button_r_label_link = traitlets.dlink((controller.buttons[5], 'value'), (buttonR_widget, 'value'))


try: axis_0_label_link
except NameError: axis_0_label_link = None

if axis_0_label_link:
    axis_0_label_link.unlink()

axis_0_label_link = traitlets.dlink((controller.axes[0], 'value'), (axes0_widget, 'value'))

try: steering_left_link
except NameError: steering_left_link = None

if steering_left_link:
    steering_left_link.unlink()

steering_left_link = traitlets.dlink((stearing, 'left_motor'), (left_motor_widget, 'value'))

try: steering_right_link
except NameError: steering_right_link = None

if steering_right_link:
    steering_right_link.unlink()
    
steering_right_link = traitlets.dlink((stearing, 'right_motor'), (right_motor_widget, 'value'))
'''

try: left_link
except NameError: left_link = None
if left_link:
    left_link.unlink()
left_link = traitlets.dlink((stearing, 'left_motor'), (robot.left_motor, 'value'))

try: right_link
except NameError: right_link = None
if right_link:
    right_link.unlink()
right_link = traitlets.dlink((stearing, 'right_motor'), (robot.right_motor, 'value'))


## Recording the training session

To record the training inputs (e.g. the camera value) and the labels (e.g. the left motor and right motor values). We use a syncronous approach. 

We create a recorder class object which takes down the camera value, the motor values and writes them into a numpy array (which will be seralized into a npy file) periodically. The FPS variable controls how often the recording should (re-)occur.

Note that for simplicity, we only record the camera value and motor values, feel free to add other values into the recording based on your need.


In [49]:
import threading, os
import time

direction = 't2_curve'

file_path = os.path.join(os.getcwd(),'data', '%s_%s' % (direction, time.strftime("%Y_%b_%d_%H_%M_%S", time.gmtime())))
FPS = 10
class Recorder(threading.Thread):
    
    writer = None
    camera = None
    left_motor = None
    right_motor = None
    axis = None
    a_button = None
    b_button = None
    is_recording = False
    fps = 10
    buffer_size = 512
    buffer = []
    def __init__(self, fps, _a_button, _b_button, _axis, _left_motor, _right_motor,  _camera, file_path):
        self.left_motor = _left_motor
        self.right_motor = _right_motor
        self.axis = _axis
        self.a_button = _a_button
        self.b_button = _b_button
        self.camera = _camera
        self.fps = fps
        threading.Thread.__init__(self)

    def run(self):
        self.is_recording = True
        print(time.time())
        self.loop()
        
    def save_buffer(self):
        file_path = os.path.join(os.getcwd(),'data', '%s_%s' % (direction, time.strftime("%Y%m%d_%H%M%S", time.gmtime())))
        np.save(file_path, self.buffer)
        self.buffer = []
    
    def loop(self):
        while self.is_recording: 
            if len(self.buffer) >= self.buffer_size:
                self.save_buffer()
            time.sleep(1/self.fps)
            image = self.camera.value
            lvalue = self.left_motor.value
            rvalue = self.right_motor.value
            self.buffer.append([lvalue, rvalue, image])
            
            
    def stop(self):
        self.is_recording = False
        print(time.time)
        if len(self.buffer) > 0:
                self.save_buffer()


### Run the following to start the recording

In [50]:
display(controller_container)

In [69]:
recorder = Recorder(FPS, controller.buttons[1], controller.buttons[2], controller.axes[0], robot.left_motor, robot.right_motor, camera, file_path)
recorder.start()

1582352959.5366938


### Run the following to stop the recording

In [70]:
recorder.stop()

<built-in function time>


In [22]:
robot.stop()